# Data Cleaning
---

## Now that we have web scraped our user reviews for the top 100 games on each console let's read in our data and combine it all into one master data frame.

In [5]:
import pandas as pd